<a href="https://colab.research.google.com/github/lucasabbruzzini/Portfolio/blob/main/DerivativePricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
#Def classes and funcs

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite, seed):
    np.random.seed(seed)
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

def cho_matrix_gen(rho):

    # Covariance Matrix
    covariance_matrix = np.zeros((2, 2), dtype=float)
    covariance_matrix[0] = [1.0, rho]
    covariance_matrix[1] = [rho, 1.0]
    cho_matrix = np.linalg.cholesky(covariance_matrix)

    return cho_matrix




def merton_paths(S0, r, sigma, mu, delta, lamb, T, M, Ite, rand):

  dt = T / M  # Define the time step

  SM = np.zeros((M + 1, Ite))
  SM[0] = S0

  # rj
  rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

  # Random numbers
  z1 = np.random.standard_normal((M + 1, Ite))
  z2 = np.random.standard_normal((M + 1, Ite))
  y = np.random.poisson(lamb * dt, (M + 1, Ite))

  for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

  return SM

def bs_call_mc(S, K, r, sigma, T, t, Ite, seed):
    np.random.seed(seed)

    data = np.zeros((Ite, 2))
    z = np.random.normal(0, 1, [1, Ite])
    # Use the prices from the last time step of Heston simulation
    ST = S * np.exp((T - t) * (r - 0.5 * sigma**2) + sigma * np.sqrt(T - t) * z)
    data[:, 1] = ST - K

    average = np.sum(np.amax(data, axis=1)) / float(Ite)

    return round(np.exp(-r * (T - t)) * average, 2)

def bs_put_mc(S, K, r, sigma, T, t, Ite, seed):
    np.random.seed(seed)

    data = np.zeros((Ite, 2))
    z = np.random.normal(0, 1, [1, Ite])
    # Use the prices from the last time step of Heston simulation
    ST = S * np.exp((T - t) * (r - 0.5 * sigma**2) + sigma * np.sqrt(T - t) * z)
    data[:, 1] = K - ST

    average = np.sum(np.amax(data, axis=1)) / float(Ite)

    return round(np.exp(-r * (T - t)) * average, 2)


def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)
    average = np.mean(payoff)
    return round(np.exp(-r * (T - t)) * average, 2)

def heston_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])
    average = np.mean(payoff)
    return round(np.exp(-r * (T - t)) * average, 2)

def merton_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return round(np.exp(-r * (T - t)) * average,2)

def merton_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])
    average = np.mean(payoff)
    return round(np.exp(-r * (T - t)) * average,2)

def bs_call_mc_american(S, K, r, sigma, T, t, Ite, seed):
    np.random.seed(seed)

    dt = T - t  # Time to maturity
    discount_factor = np.exp(-r * dt)  # Discount factor per time step

    z = np.random.normal(0, 1, [1, Ite])
    ST = S * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

    # Payoff for the call option
    payoff = np.maximum(ST - K, 0)

    # Since it's American, we consider immediate exercise at t=0
    exercise = np.maximum(S - K, 0)
    option_values = np.maximum(exercise, discount_factor * payoff)

    Opt_Price = np.mean(option_values)

    return round(Opt_Price, 2)

def bs_put_mc_american(S, K, r, sigma, T, t, Ite, seed):
    np.random.seed(seed)

    dt = T - t  # Time to maturity
    discount_factor = np.exp(-r * dt)  # Discount factor per time step

    z = np.random.normal(0, 1, [1, Ite])
    ST = S * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

    # Payoff for the put option
    payoff = np.maximum(K - ST, 0)

    # Since it's American, we consider immediate exercise at t=0
    exercise = np.maximum(K - S, 0)

    option_values = np.maximum(exercise, discount_factor * payoff)

    Opt_Price = np.mean(option_values)

    return round(Opt_Price, 2)



def heston_call_mc_american(S, K, r, T, t):
    dt = (T - t) / (S.shape[0] - 1)  # Time step size
    discount_factor = np.exp(-r * dt)

    # Payoff at maturity
    payoffs = np.maximum(S[-1, :] - K, 0)

    # Work backwards through time steps
    for i in range(len(S) - 2, -1, -1):
        # Discount future payoff
        payoffs *= discount_factor

        # Payoff if the option is exercised at this time step
        exercise_value = np.maximum(S[i, :] - K, 0)

        # Determine whether to exercise the option or continue holding it
        payoffs = np.maximum(payoffs, exercise_value)

    # Average the results across all paths
    option_price = np.mean(payoffs) * np.exp(-r * (T - t))

    return round(option_price, 2)

def heston_put_mc_american(S, K, r, T, t):

    dt = (T - t) / (S.shape[0] - 1)  # Time step size
    discount_factor = np.exp(-r * dt)

    # Payoff at maturity
    payoffs = np.maximum(K - S[-1, :], 0)

    # Work backwards through time steps
    for i in range(len(S) - 2, -1, -1):
        # Discount future payoff
        payoffs *= discount_factor

        # Payoff if the option is exercised at this time step
        exercise_value = np.maximum(K - S[i, :], 0)

        # Determine whether to exercise the option or continue holding it
        payoffs = np.maximum(payoffs, exercise_value)

    # Average the results across all paths
    option_price = np.mean(payoffs) * np.exp(-r * (T - t))

    return round(option_price, 2)

def merton_call_mc_american(S, K, r, T, t):
    dt = (T - t) / (S.shape[0] - 1)  # Time step
    discount_factor = np.exp(-r * dt)

    # Calculate payoff at maturity
    payoffs = np.maximum(S[-1, :] - K, 0)

    # Work backwards through time steps
    for i in range(len(S) - 2, -1, -1):
        # Discount future payoff
        payoffs *= discount_factor

        # Payoff if option is exercised at this time step
        exercise_value = np.maximum(S[i, :] - K, 0)

        # Decide whether to exercise or hold
        payoffs = np.maximum(payoffs, exercise_value)

    # Average the results across all paths
    option_price = np.mean(payoffs) * np.exp(-r * (T - t))

    return round(option_price, 2)

def merton_put_mc_american(S, K, r, T, t):
    dt = (T - t) / (S.shape[0] - 1)  # Time step
    discount_factor = np.exp(-r * dt)

    # Calculate payoff at maturity
    payoffs = np.maximum(K - S[-1, :], 0)

    # Work backwards through time steps
    for i in range(len(S) - 2, -1, -1):
        # Discount future payoff
        payoffs *= discount_factor

        # Payoff if option is exercised at this time step
        exercise_value = np.maximum(K - S[i, :], 0)

        # Decide whether to exercise or hold
        payoffs = np.maximum(payoffs, exercise_value)

    # Average the results across all paths
    option_price = np.mean(payoffs) * np.exp(-r * (T - t))

    return round(option_price, 2)

def greeks(S0, K, r, sigma, T, t):

  d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  Delta = ss.norm.cdf(d1)
  Gamma = ss.norm.pdf(d1) / (S0 * sigma * np.sqrt(T))
  Vega = S0 * ss.norm.pdf(d1) * np.sqrt(T)
  Theta = -(S0 * ss.norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * ss.norm.cdf(d2)
  Rho = K * T * np.exp(-r * T) * ss.norm.cdf(d2)

  print("Delta = {:.2f}".format(Delta))
  print("Gamma = {:.2f}".format(Gamma))


def uai_call_europe(S, K, r, T, t, barrier):

    # Check for barrier breach
    barrier_breached = np.any(S > barrier, axis=0)

    # Payoff calculation: Only considered paths where barrier has been breached
    payoff = np.where(barrier_breached, np.maximum(S[-1, :] - K, 0), 0)

    # Calculate the option price
    average_payoff = np.mean(payoff)
    Opt_Price = average_payoff * np.exp(-r * (T - t))

    return round(Opt_Price, 2)

def dai_put_europe(S, K, r, T, t, barrier):


    # Check for barrier breach (down-and-in condition)
    barrier_breached = np.any(S < barrier, axis=0)


    # Payoff calculation: Only consider paths where the barrier has been breached
    payoff = np.where(barrier_breached, np.maximum(K - S[-1, :], 0), 0)

    # Calculate the option price
    average_payoff = np.mean(payoff)
    Opt_Price = average_payoff * np.exp(-r * (T - t))

    return round(Opt_Price, 2)

def delta_gamma_heston(v0, kappa, theta, sigma, T, M, Ite, rand, row, S0, cho_matrix, opt, epsilon, r, K, t):
    # Volatility process paths
    V = SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix)

    # Price with the original S0
    S = Heston_paths(S0, r, V, row, cho_matrix)
    if opt == 'C':
        C_S0 = heston_call_mc(S, K, r, T, t)
    else:
        C_S0 = heston_put_mc(S, K, r, T, t)

    # Price with S0 + epsilon
    S_plus = Heston_paths(S0 + epsilon, r, V, row, cho_matrix)
    if opt == 'C':
        C_S_plus = heston_call_mc(S_plus, K, r, T, t)
    else:
        C_S_plus = heston_put_mc(S_plus, K, r, T, t)

    # Price with S0 - epsilon
    S_minus = Heston_paths(S0 - epsilon, r, V, row, cho_matrix)
    if opt == 'C':
        C_S_minus = heston_call_mc(S_minus, K, r, T, t)
    else:
        C_S_minus = heston_put_mc(S_minus, K, r, T, t)

    # Calculate Delta
    Delta = (C_S_plus - C_S_minus) / (2 * epsilon)

    # Calculate Gamma
    Gamma = (C_S_plus - 2 * C_S0 + C_S_minus) / (epsilon ** 2)

    print("Delta = {:.2f}".format(Delta))
    print("Gamma = {:.2f}".format(Gamma))
    return Delta, Gamma

# Delta and Gamma calculation for Merton model
def delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt, epsilon, K, t):
    # Price with the original S0
    S = merton_paths(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed)
    if opt == 'C':
        C_S0 = merton_call_mc(S, K, r, T, t)
    else:
        C_S0 = merton_put_mc(S, K, r, T, t)

    # Price with S0 + epsilon
    S_plus = merton_paths(S0 + epsilon, r, sigma, mu, delta, lamb, T, M, Ite, seed)
    if opt == 'C':
        C_S_plus = merton_call_mc(S_plus, K, r, T, t)
    else:
        C_S_plus = merton_put_mc(S_plus, K, r, T, t)

    # Price with S0 - epsilon
    S_minus = merton_paths(S0 - epsilon, r, sigma, mu, delta, lamb, T, M, Ite, seed)
    if opt == 'C':
        C_S_minus = merton_call_mc(S_minus, K, r, T, t)
    else:
        C_S_minus = merton_put_mc(S_minus, K, r, T, t)

    # Calculate Delta
    Delta = (C_S_plus - C_S_minus) / (2 * epsilon)

    # Calculate Gamma
    Gamma = (C_S_plus - 2 * C_S0 + C_S_minus) / (epsilon ** 2)

    print("Delta = {:.2f}".format(Delta))
    print("Gamma = {:.2f}".format(Gamma))
    return Delta, Gamma

def check_put_call_parity(S0, K, r, T, C, P):
    # Calculate RHS of the put-call parity
    rhs = S0 - K * np.exp(-r * T)
    # Calculate LHS (C - P)
    lhs = C - P
    lhs_rounded = round(lhs, 2)
    rhs_rounded = round(rhs, 2)
    parity_holds = np.isclose(lhs_rounded, rhs_rounded, rtol=0.2)

    print("LHS (rounded) = {:.2f}".format(lhs_rounded))
    print("RHS (rounded) = {:.2f}".format(rhs_rounded))
    print("Put-Call Parity: ", parity_holds)

    return parity_holds

def put_call_parity(S0, K, r, T, C, P):
    rhs = S0 - K * np.exp(-r * T)
    lhs = C - P
    lhs_rounded = round(lhs, 2)
    rhs_rounded = round(rhs, 2)
    parity_holds = np.isclose(lhs_rounded, rhs_rounded, rtol=0.2)
    return parity_holds

In [ ]:
# Initializing common parameters

S0 = 80
r = 0.05
sigma = 0.35
T = 3/12

#Heston
v0 = 0.032
kappa_v = 1.85
theta_v = 0.045

#seed
seed = 0
Ite = 100000
M = 100
epsilon = 1e-1

In [ ]:
# Question 5

rho = -0.3
K = S0
t = 0
sigma_v = sigma
dt = T / M  # T = maturity, M = number of time steps


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite, seed)

# Covariance Matrix
cho_matrix = cho_matrix_gen(rho)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
HS5 = Heston_paths(S0, r, V, 0, cho_matrix)

print("European Call Price under Heston: $", heston_call_mc(HS5, K, r, T, t))
print("European Put Price under Heston: $", heston_put_mc(HS5, K, r, T, t))

print("-" * 50)
print("European Call Price under BS (MC): $", bs_call_mc(S0, K, r, sigma, T, t, Ite, seed))
print("European Put Price under BS (MC): $", bs_put_mc(S0, K, r, sigma, T, t, Ite, seed))

# Question 7


print("-" * 50)

print('Delta and Gamma BS')
greeks(S0, K, r, sigma, T, t)

print("-" * 50)
print('Delta and Gamma for Call - Heston')
delta_gamma_heston(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, S0, cho_matrix, opt='C', epsilon=epsilon, r=r, K=K, t=t)

print("-" * 50)


print('Delta and Gamma for Put - Heston')
delta_gamma_heston(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, S0, cho_matrix, opt='P', epsilon=epsilon, r=r, K=K, t=t)


#Question 11


print("-" * 50)

print('Put-Call Parity Heston')
check_put_call_parity(S0, K, r, T, heston_call_mc(HS5, K, r, T, t), heston_put_mc(HS5, K, r, T, t))


print("-" * 50)

print('Put-Call Parity BS')
check_put_call_parity(S0, K, r, T, bs_call_mc(S0, K, r, sigma, T, t, Ite, seed), bs_put_mc(S0, K, r, sigma, T, t, Ite, seed))



European Call Price under Heston: $ 2.82
European Put Price under Heston: $ 2.87
--------------------------------------------------
European Call Price under BS (MC): $ 6.04
European Put Price under BS (MC): $ 5.03
--------------------------------------------------
Delta and Gamma BS
Delta = 0.56
Gamma = 0.03
--------------------------------------------------
Delta and Gamma for Call - Heston
Delta = 0.70
Gamma = -0.00
--------------------------------------------------
Delta and Gamma for Put - Heston
Delta = -0.30
Gamma = 0.00
--------------------------------------------------
Put-Call Parity Heston
LHS (rounded) = -0.05
RHS (rounded) = 0.99
Put-Call Parity:  False
--------------------------------------------------
Put-Call Parity BS
LHS (rounded) = 1.01
RHS (rounded) = 0.99
Put-Call Parity:  True


True

In [ ]:
# Question 6

rho = -0.7
K = S0
t = 0
sigma_v = sigma


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite, seed)

# Covariance Matrix
cho_matrix = cho_matrix_gen(rho)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)


print("European Call Price under Heston: $", heston_call_mc(S, K, r, T, t))


print("European Put Price under Heston: $", heston_put_mc(S, K, r, T, t))

print("-" * 50)

# Question 7
print('Delta and Gamma for Call')

delta_gamma_heston(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, S0, cho_matrix,opt='C', epsilon=epsilon, r=r, K=K, t=t)

print("-" * 50)

print('Delta and Gamma for Put')
delta_gamma_heston(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, S0, cho_matrix, opt='P', epsilon=epsilon, r=r, K=K, t=t)

#Question 11

print("-" * 50)

print('Put-Call Parity Heston')
check_put_call_parity(S0, K, r, T, heston_call_mc(S, K, r, T, t), heston_put_mc(S, K, r, T, t))

European Call Price under Heston: $ 2.07
European Put Price under Heston: $ 3.49
--------------------------------------------------
Delta and Gamma for Call
Delta = 0.75
Gamma = 1.00
--------------------------------------------------
Delta and Gamma for Put
Delta = -0.30
Gamma = 0.00
--------------------------------------------------
Put-Call Parity Heston
LHS (rounded) = -1.42
RHS (rounded) = 0.99
Put-Call Parity:  False


False

In [ ]:
# Question 8

lamb = 0.75
mu = -0.5  # Mu
delta = 0.22  # Delta

K = S0

SM = merton_paths(S0, r, sigma, mu, delta, lamb, T, M, Ite, rand)

print("European Call Price under Merton: ", merton_call_mc(SM, K, r, T, t))
print("European Put Price under Merton: ", merton_put_mc(SM, K, r, T, t))

# Question 10

print("-" * 50)
print('Delta and Gamma for Call')
delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt='C', epsilon=epsilon, K=K, t=t)

print("-" * 50)
print('Delta and Gamma for Put')

delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt='P', epsilon=epsilon, K=K, t=t)

# Question 11

print("-" * 50)

print('Put-Call Parity Merton')
check_put_call_parity(S0, K, r, T, merton_call_mc(SM, K, r, T, t), merton_put_mc(SM, K, r, T, t))

European Call Price under Merton:  8.24
European Put Price under Merton:  7.25
--------------------------------------------------
Delta and Gamma for Call
Delta = 1.00
Gamma = -14.00
--------------------------------------------------
Delta and Gamma for Put
Delta = -0.85
Gamma = 1.00
--------------------------------------------------
Put-Call Parity Merton
LHS (rounded) = 0.99
RHS (rounded) = 0.99
Put-Call Parity:  True


True

In [ ]:
# Question 9

lamb = 0.25
mu = -0.5  # Mu
delta = 0.22  # Delta

K = S0

SM = merton_paths(S0, r, sigma, mu, delta, lamb, T, M, Ite, rand)

print("European Call Price under Merton: ", merton_call_mc(SM, K, r, T, t))
print("European Put Price under Merton: ", merton_put_mc(SM, K, r, T, t))

# Question 10
print("-" * 50)
print('Delta and Gamma for Call')
delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt='C', epsilon=epsilon, K=K, t=t)
print("-" * 50)
print('Delta and Gamma for Put')

delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt='P', epsilon=epsilon, K=K, t=t)

# Question 11

print("-" * 50)
print('Put-Call Parity Merton')
check_put_call_parity(S0, K, r, T, merton_call_mc(SM, K, r, T, t), merton_put_mc(SM, K, r, T, t))

European Call Price under Merton:  6.75
European Put Price under Merton:  5.82
--------------------------------------------------
Delta and Gamma for Call
Delta = 0.65
Gamma = 7.00
--------------------------------------------------
Delta and Gamma for Put
Delta = -0.20
Gamma = -12.00
--------------------------------------------------
Put-Call Parity Merton
LHS (rounded) = 0.93
RHS (rounded) = 0.99
Put-Call Parity:  True


True

In [ ]:
# Question 12

# Moneyness values
moneyness_values = [0.85, 0.90, 0.95, 1, 1.05, 1.10, 1.15]
strikes = [S0 / m for m in moneyness_values]

# Initialize lists to store the results
heston_call_prices = []
merton_call_prices = []
heston_put_prices = []
merton_put_prices = []
put_call_Heston_results = []
put_call_Merton_results = []

for K in strikes:

    # Heston Model prices
    rand = random_number_gen(M, Ite, seed)
    cho_matrix = cho_matrix_gen(rho)
    V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
    S = Heston_paths(S0, r, V, 0, cho_matrix)
    heston_call_price = heston_call_mc(S, K, r, T, t)
    heston_put_price = heston_put_mc(S, K, r, T, t)
    put_call_Heston = put_call_parity(S0, K, r, T, heston_call_price, heston_put_price)

    # Merton Model prices
    SM = merton_paths(S0, r, sigma, mu, delta, lamb, T, M, Ite, rand)
    merton_call_price = merton_call_mc(SM, K, r, T, t)
    merton_put_price = merton_put_mc(SM, K, r, T, t)
    put_call_Merton = put_call_parity(S0, K, r, T, merton_call_price, merton_put_price)

    # Storing the results
    heston_call_prices.append(heston_call_price)
    heston_put_prices.append(heston_put_price)
    merton_call_prices.append(merton_call_price)
    merton_put_prices.append(merton_put_price)
    put_call_Heston_results.append(put_call_Heston)
    put_call_Merton_results.append(put_call_Merton)

# Display the results
for i, K in enumerate(strikes):
    print(f"Strike: {K:.2f}")
    print(f"Heston Call: {heston_call_prices[i]:.4f}, Heston Put: {heston_put_prices[i]:.4f}, Put-Call: {put_call_Heston_results[i]}")
    print(f"Merton Call: {merton_call_prices[i]:.4f}, Merton Put: {merton_put_prices[i]:.4f}, Put-Call: {put_call_Merton_results[i]}")
    print("-" * 100)

Strike: 94.12
Heston Call: 0.0100, Heston Put: 15.3700, Put-Call: True
Merton Call: 1.9500, Merton Put: 14.9300, Put-Call: True
----------------------------------------------------------------------------------------------------
Strike: 88.89
Heston Call: 0.1200, Heston Put: 10.3200, Put-Call: False
Merton Call: 3.2100, Merton Put: 11.0200, Put-Call: True
----------------------------------------------------------------------------------------------------
Strike: 84.21
Heston Call: 0.6800, Heston Put: 6.2600, Put-Call: False
Merton Call: 4.8200, Merton Put: 8.0100, Put-Call: True
----------------------------------------------------------------------------------------------------
Strike: 80.00
Heston Call: 2.0700, Heston Put: 3.4900, Put-Call: False
Merton Call: 6.7400, Merton Put: 5.7700, Put-Call: True
----------------------------------------------------------------------------------------------------
Strike: 76.19
Heston Call: 4.2000, Heston Put: 1.8500, Put-Call: False
Merton Call: 8

In [ ]:
# Question 13

rho = -0.3
K = S0
t = 0
sigma_v = sigma


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite, seed)

# Covariance Matrix
cho_matrix = cho_matrix_gen(rho)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)


print("American Call Price under Heston: $", heston_call_mc_american(S, K, r, T, t))
print("American Put Price under Heston: $", heston_put_mc_american(S, K, r, T, t))
print("-" * 50)

print("American Call Price under BS (MC): $", bs_call_mc_american(S0, K, r, sigma, T, t, Ite, seed))
print("American Put Price under BS (MC): $", bs_put_mc_american(S0, K, r, sigma, T, t, Ite, seed))

#Question 11


print("-" * 50)

print('Put-Call Parity Heston')
check_put_call_parity(S0, K, r, T, heston_call_mc(S, K, r, T, t), heston_put_mc(S, K, r, T, t))


print("-" * 50)

print('Put-Call Parity BS')
check_put_call_parity(S0, K, r, T, bs_call_mc(S0, K, r, sigma, T, t, Ite, seed), bs_put_mc(S0, K, r, sigma, T, t, Ite, seed))

American Call Price under Heston: $ 5.17
American Put Price under Heston: $ 5.28
--------------------------------------------------
American Call Price under BS (MC): $ 6.04
American Put Price under BS (MC): $ 5.03
--------------------------------------------------
Put-Call Parity Heston
LHS (rounded) = -0.05
RHS (rounded) = 0.99
Put-Call Parity:  False
--------------------------------------------------
Put-Call Parity BS
LHS (rounded) = 1.01
RHS (rounded) = 0.99
Put-Call Parity:  True


True

In [ ]:
# Question 13 cont

lamb = 0.75
mu = -0.5  # Mu
delta = 0.22  # Delta

K = S0

SM = merton_paths(S0, r, sigma, mu, delta, lamb, T, M, Ite, rand)

print("American Call Price under Merton: ", merton_call_mc_american(SM, K, r, T, t))
print("American Put Price under Merton: ", merton_put_mc_american(SM, K, r, T, t))

print("-" * 50)

print('Delta and Gamma for Call')
delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt='C', epsilon=epsilon, K=K, t=t)
print("-" * 50)

print('Delta and Gamma for Put')

delta_gamma_merton(S0, r, sigma, mu, delta, lamb, T, M, Ite, seed, opt='P', epsilon=epsilon, K=K, t=t)


print("-" * 50)

print('Put-Call Parity Merton')
check_put_call_parity(S0, K, r, T, merton_call_mc(SM, K, r, T, t), merton_put_mc(SM, K, r, T, t))


American Call Price under Merton:  13.96
American Put Price under Merton:  11.43
--------------------------------------------------
Delta and Gamma for Call
Delta = 0.70
Gamma = -2.00
--------------------------------------------------
Delta and Gamma for Put
Delta = 0.25
Gamma = -5.00
--------------------------------------------------
Put-Call Parity Merton
LHS (rounded) = 1.08
RHS (rounded) = 0.99
Put-Call Parity:  True


True

In [ ]:
# Question 14

rho = -0.7
K = 95
t = 0
sigma_v = sigma
barrier = 95
S14 = 95


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite, seed)

# Covariance Matrix
cho_matrix = cho_matrix_gen(rho)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S14, r, V, 0, cho_matrix)

print("European Call Price under Heston: $", heston_call_mc(S, K, r, T, t))
print("European Call Price under Heston UAI: $", uai_call_europe(S, K, r, T, t, barrier))

S14 = 100


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite, seed)

# Covariance Matrix
cho_matrix = cho_matrix_gen(rho)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S14, r, V, 0, cho_matrix)

print("European Call Price under Heston: $", heston_call_mc(S, K, r, T, t))
print("European Call Price under Heston UAI: $", uai_call_europe(S, K, r, T, t, barrier))


European Call Price under Heston: $ 2.46
European Call Price under Heston UAI: $ 2.46
European Call Price under Heston: $ 5.4
European Call Price under Heston UAI: $ 5.4


In [ ]:
# Question 15

lamb = 0.75
mu = -0.5  # Mu
delta = 0.22  # Delta
barrier = 65
S15 = 60
K = 65


SM = merton_paths(S15, r, sigma, mu, delta, lamb, T, M, Ite, rand)



print("European Put Price under Merton DAI: $", uai_call_europe(SM, K, r, T, t, barrier))
print("European Put Price under Merton: ", merton_put_mc(SM, K, r, T, t))

S = 70

SM = merton_paths(S15, r, sigma, mu, delta, lamb, T, M, Ite, rand)

print("European Put Price under Merton DAI: $", uai_call_europe(SM, K, r, T, t, barrier))
print("European Put Price under Merton: ", merton_put_mc(SM, K, r, T, t))

European Put Price under Merton DAI: $ 3.84
European Put Price under Merton:  8.04
European Put Price under Merton DAI: $ 3.87
European Put Price under Merton:  8.03
